## Day 8

https://adventofcode.com/2021/day/8

In [1]:
def readInput8(filename):
    with open(filename) as f:
        return [ [ p.split() for p in l.strip('\n').split(" | ") ] for l in f.readlines() ]

i0 = readInput8("data/day08test01.txt")
i1 = readInput8("data/day08test02.txt")
i2 = readInput8("data/input08.txt")

### Part 1
Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Count only digits in the output values.

In [2]:
from collections import Counter

def countUnique(o):
    # 1 = 2S ; 4 = 4S ; 7 = 3S ; 8 = 7S
    c = Counter([ len(d) for d in o ])
    return c[2]+c[4]+c[3]+c[7]

def part1(i0):
    return sum([ countUnique(o) for i,o in i0 ])

print("Test 1:",part1(i1))
print("Part 1:",part1(i2))

Test 1: 26
Part 1: 512


### Part 2 brute force

Not trying to be smart, probing all wiring permutations for each line.

In [55]:
from itertools import permutations

display = {}
display["abcefg"]  = 0
display["cf"]      = 1
display["acdeg"]   = 2
display["acdfg"]   = 3
display["bcdf"]    = 4
display["abdfg"]   = 5
display["abdefg"]  = 6
display["acf"]     = 7
display["abcdefg"] = 8
display["abcdfg"]  = 9

def checkWiring(di):
    return sum([1 if i in display.keys() else 0 for i in di ])==len(di)

def part2(i0,verbose=True):
    output = 0
    origin = 'abcdefg'
    do = ""
    for i,o in i0:
        # find correct wiring for current line
        for p in permutations(origin):
            scramb = "".join(p)
            # generate wiring for given permutation
            wiring = {}
            for ori,scr in zip(origin,scramb):
                wiring[scr] = ori
            # test wiring on line
            di = [ "".join(sorted([ wiring[s] for s in ii ])) for ii in i ]
            do = [ "".join(sorted([ wiring[s] for s in oo ])) for oo in o ]
            dd = list(dict.fromkeys(di+do)) # remove duplicates
            if checkWiring(dd): break
        # output value
        value = int("".join(str(display[oo]) for oo in do))
        if verbose: 
            for _o in o:
                print("{:7s} ".format(_o),end=" ")
            print(value)
        output += value
    return output

In [56]:
part2(i0)

cdfeb    fcadb    cdfeb    cdbaf    5353


5353

In [57]:
part2(i1)

fdgacbe  cefdb    cefbgd   gcbe     8394
fcgedb   cgb      dgebacf  gc       9781
cg       cg       fdcagb   cbg      1197
efabcd   cedba    gadfec   cb       9361
gecf     egdcabf  bgf      bfgea    4873
gebdcfa  ecba     ca       fadegcb  8418
cefg     dcbef    fcge     gbcadfe  4548
ed       bcgafe   cdgba    cbgef    1625
gbdfcae  bgc      cg       cgb      8717
fgae     cfgab    fg       bagce    4315


61229

In [58]:
part2(i2,False)

1091165

### Part 2 clever

I can use univoque correspondence of segments of unique lenghts (corresponding to 1, 4, 7 and 8) to reduce the search space, and use them to initialize the decoding dictionary

In [60]:
def checkWiring(di):
    return sum([1 if i in display.keys() else 0 for i in di ])==len(di)

def part2fast(i0,verbose=True):
    output = 0
    origin = 'abcdefg'
    do = ""
    for i,o in i0:
        foundWiring = False
        # remove duplicates
        io = list(dict.fromkeys(i+o))
        # find unique segments
        io_len = [ len(s) for s in io]
        s1 = io[io_len.index(2)]
        s7 = "".join([c for c in io[io_len.index(3)] if c not in s1 ])
        s4 = "".join([c for c in io[io_len.index(4)] if c not in s1 ])
        for p1 in permutations(s1): # permutations for 1 segments (cf)
            w1 = {}
            for ori,scr in zip('cf',p1):
                w1[scr] = ori
            # add permutations from 4 segments ('bd')
            for p4 in permutations(s4): 
                w4 = w1.copy()
                for ori,scr in zip('bd',p4):
                    w4[scr] = ori
                # add third segment from 7 ('a')
                w4[s7] = 'a'      
                # add permutations for remaining segments ('eg')
                origin = 'eg'
                scramb = ''.join([ c for c in 'abcdefg' if c not in w4.keys() ])                
                for p in permutations(scramb):
                    wiring = w4.copy()
                    for ori,scr in zip(origin,p):
                        wiring[scr] = ori
                    # test wiring on line
                    di = [ "".join(sorted([ wiring[s] for s in ii ])) for ii in i ]
                    do = [ "".join(sorted([ wiring[s] for s in oo ])) for oo in o ]
                    dd = list(dict.fromkeys(di+do)) # remove duplicates
                    if checkWiring(dd):
                        foundWiring = True
                        break
                if foundWiring: break
            if foundWiring: break
        
        if foundWiring:
            # output value
            value = int("".join(str(display[oo]) for oo in do))
            if verbose: 
                for _o in o:
                    print("{:7s} ".format(_o),end=" ")
                print(value)
            output += value
        else:
            print("ERROR")
            return -1
    return output

In [61]:
part2fast(i0)

cdfeb    fcadb    cdfeb    cdbaf    5353


5353

In [62]:
part2fast(i1)

fdgacbe  cefdb    cefbgd   gcbe     8394
fcgedb   cgb      dgebacf  gc       9781
cg       cg       fdcagb   cbg      1197
efabcd   cedba    gadfec   cb       9361
gecf     egdcabf  bgf      bfgea    4873
gebdcfa  ecba     ca       fadegcb  8418
cefg     dcbef    fcge     gbcadfe  4548
ed       bcgafe   cdgba    cbgef    1625
gbdfcae  bgc      cg       cgb      8717
fgae     cfgab    fg       bagce    4315


61229

In [63]:
part2fast(i2,False)

1091165